In [10]:
from gurobipy import *
import csv
import matplotlib.pyplot as plt
import folium

"""
:param N: 所有节点集合,其中N[0]为车场
:param Q: 节点需求集合
:param TT: 旅行时间
:param ET: 节点最早开始服务时间
:param LT:节点最晚结束服务时间
:param ST: 节点服务时间
:param CAP: 车辆容量
:param Cost: 旅行费用
:param K: 车队数量(可自由调节)
"""

N=[]  #所有节点
Q={}  #节点需求
TT={} #节点旅行时间
ET={} #节点最早开始服务时间
LT={} #节点最晚结束服务时间
ST={} #节点服务时间
Cost={}

with open('nodes.csv','r') as f:
    node_reader = csv.DictReader(f)
    for row in node_reader:
        node_id = row['id']
        demand = float(row['demand'])
        start_time = float(row['start_time'])
        end_time = float(row['end_time'])
        service_time = float(row['service_time'])
        N.append(node_id)
        Q[node_id] = demand
        ET[node_id] = start_time
        LT[node_id] = end_time
        ST[node_id] = service_time
with open('links.csv','r') as f:
    link_reader = csv.DictReader(f)
    for row in link_reader:
        link_node_id = row['from_node_id']
        to_node_id = row['to_node_id']        
        travel_time = float(row['travel_time'])
        travel_cost = float(row['link_cost'])
        TT[link_node_id,to_node_id] = travel_time
        Cost[link_node_id,to_node_id] = travel_cost

CAP = 80           #容量约束
K = 15             #车队数量
C=tuplelist(N[1:]) #需求节点
N=tuplelist(N)
Q=tupledict(Q)
TT=tupledict(TT)
E=tupledict(ET)
L=tupledict(LT)
S=tupledict(ST)
K=tuplelist([f'v'+str(i) for i in range(K)])
M=10**5
depot = N[0]

In [11]:
m = Model('cvrptw')

x = m.addVars(N,N,K, vtype=GRB.BINARY, name='X(i,j,k)')
t = m.addVars(N,K, vtype=GRB.CONTINUOUS, lb=0, name='T[i,k]')

#设置目标函数
z1 = quicksum(Cost[i,j]*x[i,j,k] for i in N for j in N for k in K if i!=j)
m.setObjective(z1, GRB.MINIMIZE)

#车辆起点约束:让车辆驶出仓库(depot)
m.addConstrs(quicksum(x[depot, j, k] for j in N) == 1 for k in K)

#车辆路径连续约束:流平衡(除去depot之外的点)
m.addConstrs(quicksum(x[i,j,k] for j in N if j!=i) == quicksum(x[j,i,k] for j in N if j!=i) for i in C for k in K)

#车辆终点约束:让车辆驶回depot
m.addConstrs(quicksum(x[j, depot, k] for j in N) == 1 for k in K)

#需求服务约束:保证每个客户点都被服务
m.addConstrs(quicksum(x[i,j,k] for k in K for j in N if j!=i) == 1 for i in C)

#车辆容量约束:不超过CAP=80
m.addConstrs(quicksum(Q[i]*x[i,j,k] for i in C for j in N if i!=j) <= CAP for k in K )

#时间窗约束:
#保证被服务的相邻节点开始服务时间的大小关系(去回路)
m.addConstrs(t[i,k]+S[i]+TT[i,j]-(1-x[i,j,k])*M <= t[j,k] for i in C for j in C for k in K if i!=j )
#不违反客户的时间窗
m.addConstrs(t[i,k] >= E[i] for i in N for k in K)
m.addConstrs(t[i,k] <= L[i] for i in N for k in K)

m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 11021 rows, 11340 columns and 71430 nonzeros
Model fingerprint: 0xf89cc884
Variable types: 405 continuous, 10935 integer (10935 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 8475 rows and 4875 columns
Presolve time: 0.14s
Presolved: 2546 rows, 6465 columns, 30975 nonzeros
Variable types: 390 continuous, 6075 integer (6075 binary)
Found heuristic solution: objective 319.0000000

Root relaxation: objective 1.720000e+02, 1247 iterations, 0.07 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  172.00000    0   58  319.00000  172.00000  46.1%     -    0s
H    0     

In [7]:
def saveFile(data):
    outfile = open('resultstest.csv', 'w', newline='', errors='ignore')
    write = csv.writer(outfile)
    write.writerow(['from_node_id', 'to_node_id', 'vehicle','Ti','Tj'])
    for v in data:
        write.writerow(v)
    outfile.close()

if m.status == GRB.OPTIMAL:
    print('obj={}'.format(m.objVal))
    res=[]
    for k in K:
        for i in N:
            for j in N:
                if i!=j:
                    if x[i,j,k].x>0.9:
                        print("X[{},{},{}]=1".format(i,j,k))
                        res.append([i,j,k,t[i,k].x,t[j,k].x])
    saveFile(res)
else:
    print("no solution")

obj=185.0
X[0,10,v0]=1
X[10,12,v0]=1
X[12,26,v0]=1
X[26,0,v0]=1
X[0,15,v1]=1
X[11,0,v1]=1
X[15,11,v1]=1
X[0,3,v2]=1
X[1,0,v2]=1
X[3,5,v2]=1
X[5,14,v2]=1
X[14,22,v2]=1
X[21,1,v2]=1
X[22,21,v2]=1
X[0,16,v4]=1
X[6,0,v4]=1
X[16,6,v4]=1
X[0,24,v9]=1
X[4,23,v9]=1
X[7,8,v9]=1
X[8,4,v9]=1
X[20,7,v9]=1
X[23,0,v9]=1
X[24,20,v9]=1
X[0,17,v12]=1
X[9,0,v12]=1
X[13,18,v12]=1
X[17,13,v12]=1
X[18,9,v12]=1
X[0,25,v14]=1
X[2,0,v14]=1
X[19,2,v14]=1
X[25,19,v14]=1
